# Symbolic computation with `sympy`

Symbolic (as opposed to numeric) computation can be accomplished in Python using the `sympy` package. The `init_printing` function allows the output to be formatted using *MathJax*.

In [ ]:
from sympy import init_printing, Symbol, symbols, Function, sin, cos, tan, exp, oo

init_printing()

We must explicitly state which Python variables correspond to mathematical symbols:

In [ ]:
x = Symbol('x')

sin(x+1)/exp(x)

In [ ]:
t0 = Symbol('theta_0')

sin(t0+1)/exp(t0)

In the above, `theta_0` produces the symbol $\theta_0$ in *LaTeX*. `sympy` understands this and displays the symbol correctly.

The `symbols` function (note the lowercase "s") allows us to define several symbols at once:

In [ ]:
a, b, c, x, y, z = symbols('a, b, c, x, y, z')

# Solving equations

A symbolic expression (equal to zero) can be solved for any one of the symbols it contains using the `solve` function:

In [ ]:
from sympy import solve

solve(a*x**2 + b*x + c, x)

By default, `solve` presents its solutions as a list, but a dictionary presentation is also available:

In [ ]:
solve(a*x**2 + b*x + c, x, dict=True)

We can also solve a system of equations:

In [ ]:
solve([x + y - 2*z, y**2 - 4*z], [x, y], dict=True)

# Calculus

In [ ]:
from sympy import diff, integrate

To take derivatives, we use the `diff` function, specifying the variable we wish to differentiate with respect to:

In [ ]:
diff(cos(x), x)

To take the second derivative, we differentiate with respect to $x$ twice:

In [ ]:
diff(cos(x), x, x)

Integration is similar:

In [ ]:
integrate(cos(x), x)

Definite integrals are performed as follows. Let's compute

$$
    \int_0^\infty e^{-x}\,dx.
$$

In [ ]:
integrate(exp(-x), (x, 0, oo))

**Note:** Sympy uses a double lowercase o ("oh") to denote infinity, as seen above.

# Differential equations

In [ ]:
from sympy import Derivative, dsolve

Differential equations can be solved directly (`sympy` uses several methods under the hood).

First, we must specify that $f$ is some (as yet unknown) function:

In [ ]:
f = Function('f')
t = Symbol('t')

Now, `Derivative(f(t), t)` and `Derivative(f(t), t, t)` represent the first and second derivatives, respectively.

In [ ]:
Derivative(f(t), t)

In [ ]:
Derivative(f(t), t, t)

Alternatively, you can use `f(t).diff(t)` and `f(t).diff(t, t)`.

In [ ]:
f(t).diff(t)

In [ ]:
f(t).diff(t, t)

Let's solve the second-order differential equation
$$
    f''(t)-f'(t)-6f(t)=e^t.
$$
for $f(t)$ using the `dsolve` function.

In [ ]:
dsolve(Derivative(f(t), t, t) - Derivative(f(t), t) - 6*f(t) - exp(t), f(t))

Now suppose we want to find the particular solution satisfying the initial conditions
$$
    f(0)=f'(0)=0.
$$

In [ ]:
conditions = {f(0): 0, f(t).diff(t).subs(t, 0): 0} # a Python dictionary

dsolve(Derivative(f(t), t, t) - Derivative(f(t), t) - 6*f(t) - exp(t), f(t), ics=conditions)

**Note:** In the above, `.subs(t, 0)` substituted the value 0 into the mathematical expression for $f'(t)$.

Actually, we can use `subs` more generally:

In [ ]:
F = x**2 + x + 2
F

In [ ]:
F.subs(x, t)

In [ ]:
F.subs(x, 4)

# Shot put problem revisited

Recall that the trajectory of a projectile fired from the origin and subject to earth's gravity is given by
$$
    y=(\tan\theta_0)x-\frac{gx^2}{2(v_0\cos\theta_0)^2},
$$
where $v_0$ is the launch speed and $\theta_0$ is the launch angle (above the horizontal). When launched from the ground and neglecting air resistance, the horizontal distance traveled by the projectile (its *range*) was maximised by taking $\theta_0=45^\circ$.

Let us now compute the optimal launch angle when the projectile is fired from a height $h$ above the ground. If the projectile is fired from the origin, the point of impact with the ground is given by $(x,y)=(R,-h)$. Hence, the range $R$ can be determined from the equation
$$
    0=h+(\tan\theta_0)R-\frac{gR^2}{2(v_0\cos\theta_0)^2}.\tag{1}
$$
Considering (1) as the (implicit) definition of the function $R(\theta_0)$, we can differentiate both sides with respect to $\theta_0$. In order for `sympy` to perform the implicit differentiation correctly, we must specify that $R$ is a function of $\theta_0$.

In [ ]:
h, v0, theta0, g = symbols('h, v_0, theta_0, g')
R = Function('R')(theta0)

RHS = h + tan(theta0)*R - g*R**2/(2*(v0*cos(theta0))**2)
RHS

In [ ]:
d = diff(RHS, theta0)
d

In [ ]:
d_R_d_theta0 = solve(d, R.diff(theta0), dict=True) # list containing only one solution
d_R_d_theta0

Let's store the above expression on its own, rather than wrapped in a dictionary and a list:

In [ ]:
d_R_d_theta0 = solve(d, R.diff(theta0))[0] # list containing only one solution
d_R_d_theta0

We maximise the range by setting the above derivative equal to zero.

In [ ]:
R_sols = solve(d_R_d_theta0, R)
R_sols

So, for maximum $R(\theta_0)$,
$$
    R(\theta_0)=\frac{v_0^2}{g\tan{\theta_0}}.
$$
Let's sub this back into (1) and then solve for $\theta_0$:

In [ ]:
R_sol = R_sols[1]
rhs = RHS.subs(R, R_sol).simplify()
rhs

In [ ]:
sols = solve(rhs, theta0)
sols

Sanity check: when $h=0$, we have $\theta_0=\tan^{-1}(1)=45^\circ$, so we take the latter solution:

In [ ]:
sol = sols[1]
sol

**Observe:** the taller the athlete, the smaller the optimal launch angle. Let's plug in some realistic numbers for Olympic shot put:

In [ ]:
import math

# show answer in degrees, rather than radians:
math.degrees(sol.subs([(h, 1.8), (v0, 38), (g, 9.81)]))

As we can see, the height of the athlete does not greatly impact the optimal launch angle.

# Problems

## Problem 1

Use `sympy` to derive all five equations of motion for constant acceleration (recall that we have already done this by hand in class).

## Problem 2

Let us now consider the impact of air resistance on the optimal launch angle for the shot put. Suppose, as above, that the shot put is launched from a height of $h=1.8$ m above the ground with speed $v_0=38$ m s$^{-1}$ at an angle $\theta_0$ above the horizontal. Suppose the shot put has a mass of 4 kg, is 10 cm in diameter and has a drag coefficient of $C=0.47$. Assume the density of air is 1.225 kg m$^{-3}$.

**(a)** Choose a few different values of $\theta_0$, and, for each one, numerically compute the trajectory of the shot put (using the `odeint` function). Plot the trajectories on the same graph and compute the approximate range for each trajectory.

**(b)** [*Difficult problem*] Programmatically compute the range for many different values of $\theta_0$ and hence find an approximation for the optimal $\theta_0$.